# Proyecto SQL

El coronavirus tomó al mundo por sorpresa, cambiando la rutina diaria de todos y todas. Lo atrajo la atención de las startups que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros que estaban en casa buscando materiales de lectura sin tener que salir.

Contamos con una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

### Objetivos

#### Objetivo general
Contar con información util para la generación de una propuesta de valor para un nuevo producto.

#### Objetivos específicos
Encontrar el número de libros publicados después del 1 de enero de 2000.
Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.
Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.
Identificar al autor que tiene la más alta calificación promedio del libro.
Encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

## Conectarse a la base de datos

In [2]:
#librerias
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com', 
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 
db_config['pwd'],
 
db_config['host'],
db_config['port'],
 
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Visualización de datos

In [3]:
query = "SELECT * FROM books LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = "SELECT * FROM authors LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = "SELECT * FROM publishers LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = "SELECT * FROM ratings LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = "SELECT * FROM reviews LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Consultas SQL

### Encuentra el número de libros publicados después del 1 de enero de 2000.


In [8]:
#definimos la consulta
query = """
SELECT COUNT(*) AS num_books
FROM books
WHERE publication_date > '2000-01-01';
"""

#ejecutamos la consulta
pd.read_sql(query, con=engine)


,num_books
0,819


### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.


In [9]:
#definimos la consulta
query = """
SELECT
    books.book_id,
    books.title,
    COUNT(reviews.review_id) AS num_reviews,
    AVG(ratings.rating) AS avg_rating
FROM
    books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    books.book_id, books.title;
"""
#ejecución de consulta
pd.read_sql(query, con=engine)

,book_id,title,num_reviews,avg_rating
0,652,The Body in the Library (Miss Marple #3),4,4.500000
1,273,Galápagos,4,4.500000
2,51,A Tree Grows in Brooklyn,60,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,4,4.000000
4,839,The Prophet,28,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,52,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),4,5.000000
997,148,Christine,21,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),4,3.500000


### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas.
(esto te ayudará a excluir folletos y publicaciones similares de tu análisis)


In [10]:
#definimos la consulta
query = """
SELECT 
    publishers.publisher,
    COUNT(books.book_id) AS num_books
FROM 
    books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE 
    books.num_pages > 50
GROUP BY 
    publishers.publisher
ORDER BY 
    num_books DESC
LIMIT 1;
"""
#ejecutamos la consulta
pd.read_sql(query, con=engine)

,publisher,num_books
0,Penguin Books,42


### Identifica al autor que tiene la más alta calificación promedio del libro: 
mira solo los libros con al menos 50 calificaciones.


In [11]:
#definimos la consulta
query = """
SELECT 
    authors.author,
    AVG(ratings.rating) AS avg_rating
FROM 
    books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY 
    authors.author
HAVING 
    COUNT(ratings.rating) >= 50
ORDER BY 
    avg_rating DESC
LIMIT 1;
"""
#ejecutamos
pd.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [12]:
#definición de consulta
query = """
WITH users_with_more_than_50_ratings AS (
    SELECT 
        username,
        COUNT(*) AS num_ratings
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(*) > 50
)
SELECT 
    AVG(num_reviews) AS avg_reviews_per_user
FROM (
    SELECT 
        reviews.username,
        COUNT(reviews.review_id) AS num_reviews
    FROM 
        reviews
    JOIN users_with_more_than_50_ratings 
    ON reviews.username = users_with_more_than_50_ratings.username
    GROUP BY 
        reviews.username
) AS user_reviews;"""

#ejecutar consulta
pd.read_sql(query, con=engine)

,avg_reviews_per_user
0,24.333333


## Conclusiones

#### Número de libros
Es una consulta útil para identificar los libros en el mercado a partir de la fecha de interés, lo que refleja el crecimiento del catálogo más actual para los lectores. Entre mayor es el número, mayor es la industria activa.

#### Número de reseñas
Con esta consulta podemos ver la aceptación de los libros en el mercado, los libros con mejores reseñas nos sirven para darlos en recomendaicón o promociones ya que tienen buena aceptación en el público y pueden ser los más buscados por ser más populares.

#### Editorial con más publicaciones
Esta consulta nos permite saber qué editorial tiene más publicaciones (específicamente para libros más extensos) lo que es un mercado amplio ya que suele ser material de lectura más robusto y tiene un nucho del mercado más comprometido al ser obras más extensas. Con estas editoriales pueden llegarse a acuerdos particulares.

#### Autor con la calificación más alta
Nos permite saber qué autores son los más populares y mejor aceptados entre los lectores, haciéndonos apostar con mayor seguridad por obras que tendrán mayor alcance entre sus seguidores. 

#### Número promedio de reseñas
Nos ayuda a identificar a los usuarios más activos en dar retroalimentación a las publicaciones, entre más reseñas de un usuario, más comprometido está con la comunidad de lectores. El número de reseñas nos permite saber la participación de los usuarios.


### Conclusión final
Las consultas en SQL a nuestra base de datos son utiles para obtener información sobre distintas categorías de interés, tal como los autores más leidos, las temporadas de mayor demanda y las editoriales más populares. Esta información nos da un panorama de las tendencias actuales y las preferencias de los uaurios, por lo que puede usarse para tomar decisiones de marketing y promociones. Por ejemplo, adquirir más libros de la editorial Penguin books ya que son una editorial con mucha diversidad o hacer promoción de los libros de la autora Diana Gabaldoin ya que sus libros tienen muchas reseñas lo que nos dice es conocida.
